# Determin Payer formulaires.
### v3.0

Let's start simple and see if we can determine which drugs are in a payer's formulary (reject code for formularies is 70)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('../data/dim_claims.csv')


In [3]:
#Use a 75% sample as training set.
df_train = df.sample(frac = .75)

#Create list of payers and drugs
payers = df_train['bin'].unique()
drugs = df_train['drug'].unique()

Create a new dataframe to hold the rates for each payer, drug, code combination. We impute -1 into the rejection code column to indicate the pharmacy claim was accepted.

In [4]:
df_code= pd.DataFrame(columns=drugs)

df_code.insert(loc=0,column='Payer',value=0)
df_code.insert(loc=1,column='Acc/Code',value=0)


df_train['reject_code'] = df_train['reject_code'].fillna(-1)
codes = df_train['reject_code'].unique()

rep = len(codes)

p = payers.repeat(rep)

df_code['Payer'] = p
 
#pd.DataFrame([[payer for payer in payers],[1]], columns=['Payer','Acc/Code'])
for payer in payers:
    df_code.loc[df_code['Payer'] == payer,['Acc/Code']] = codes
    


Now, fill the table with calculated rates.

In [5]:

i = 0

#df_code['Payer'] = payers

for payer in payers:
    for drug in drugs:
        for code in codes:
            den = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug)].shape[0]
        #print("Den = ",den)
            if(code>0):
                rej = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) 
                               & (df_train['pharmacy_claim_approved']==0) & (df_train['reject_code']==code)].shape[0]
            else:
                rej = 0    
            acc = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==1)].shape[0]
            rejfrac = rej/den
            accfrac = acc/den
            if(code>0):
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = rejfrac
            else:
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = accfrac
        
df_code.head(25)   



,Payer,Acc/Code,A,B,C
0,417614,70.0,1.000000,0.000000,0.000000
1,417614,75.0,0.000000,1.000000,0.000000
2,417614,-1.0,0.000000,0.000000,0.902440
3,417614,76.0,0.000000,0.000000,0.097560
4,999001,70.0,0.000000,0.000000,0.000000
5,999001,75.0,0.000000,0.000000,0.000000
6,999001,-1.0,0.900340,0.901252,0.899175
7,999001,76.0,0.099660,0.098748,0.100825
8,417740,70.0,0.000000,1.000000,0.000000
9,417740,75.0,0.000000,0.000000,1.000000



999001: No drug is rejeced due to formulay or requires prior authorization. All druges are rejected due to code 76

417614: Drug A is not in formulary (always rejected, always due to code 70)
        Drug B is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug C is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        
417740: Drug A is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug B is not in formulary (always rejected, always due to code 70)
        Drug C is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        
417380: Drug A is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug B is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug C is not in formulary (always rejected, always due to code 70)

In [7]:
df_code.to_csv("../streamlined/formulary.csv",index=False) #Output the table to a csv file so it can be used as a lookup-table for the web app.